In [1]:
from bts.mmformer.predict import val_epoch
from bts.mmformer.mmformer.mmformer import Model
from bts.data.porcupine_dataset import PorcupineDataset

import torch
from torch.utils.data import DataLoader
from torch.optim import Adam

from monai.transforms import (
    Compose,
    LoadImaged,
    MapTransform,
    NormalizeIntensityd,
    RandFlipd,
    RandSpatialCropd,
    Resized,
    Transform,
)
from bts.data.utils import UnsqueezeDatad
from bts.data.dataset import ConvertToMultiChannelBasedOnEchidnaClassesd

In [2]:
dataset_path = "/home/desmin/data/porcupine_dataset/train"

train_dataset = PorcupineDataset(dataset_root_path=dataset_path, transform=Compose([
    LoadImaged(["image"], reader="NrrdReader"),
    LoadImaged(["label"], reader="NumpyReader"),
    UnsqueezeDatad(["image", "label"]),
    RandSpatialCropd(
        keys=["image", "label"],
        roi_size=[128, 128, 128],
        random_size=False,
    ),
    Resized(
        keys=["image", "label"],
        spatial_size=[128,128,128]
    )
]))
train_loader = DataLoader(train_dataset, batch_size=1, pin_memory=True)

In [3]:
dataset_path = "/home/desmin/data/porcupine_dataset/validation"

val_dataset = PorcupineDataset(dataset_root_path=dataset_path, transform=Compose([
    LoadImaged(["image"], reader="NrrdReader"),
    LoadImaged(["label"], reader="NumpyReader"),
    UnsqueezeDatad(["image", "label"])
]))
val_loader = DataLoader(val_dataset, batch_size=1, pin_memory=True)

```
# load model
class_num = 4
weights_path = "/home/desmin/grad_project/bbm47980_bts/notebooks/mmformer_model_weights.pth"

mmformer = Model(num_cls=class_num)
mmformer.load_state_dict(torch.load(weights_path))
mmformer_parallel = torch.nn.DataParallel(mmformer).cuda()
```

In [8]:
class_num = 4
mmformer = Model(num_cls=class_num)
mmformer_woft = torch.nn.DataParallel(mmformer).cuda()

weights_path = "/home/desmin/grad_project/bbm47980_bts/bts/mmformer/mmformer/out/model_last.pth"
mmformer_woft.load_state_dict(torch.load(weights_path)["state_dict"])

<All keys matched successfully>

In [11]:
val_epoch(
    model = mmformer_woft,
    loader = train_loader,
    epoch = 0
)

  0%|          | 0/169 [00:00<?, ?it/s]

{'avg_dice': array([0.3467529], dtype=float32)}

In [12]:
val_epoch(
    model = mmformer_woft,
    loader = val_loader,
    epoch = 0
)

  0%|          | 0/20 [00:00<?, ?it/s]

{'avg_dice': array([0.17493221], dtype=float32)}